In [ ]:
#https://github.com/openai/whisper
#https://huggingface.co/openai/whisper-small/blob/main/vocab.json

!pip install onnx
!pip install onnxruntime
!pip install torch

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
%mkdir UDaySeoul2024
%cd /content/drive/MyDrive/UDaySeoul2024

!pip install git+https://github.com/openai/whisper.git
%mkdir whisper
%cd whisper

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
mkdir: cannot create directory ‘UDaySeoul2024’: File exists
/content/drive/MyDrive/UDaySeoul2024
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-jnvdyzl4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-jnvdyzl4
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
/content/drive/MyDrive/UDaySeoul2024/whisper


In [ ]:
import time
import whisper
import torch

# allow to inject real data
if "load_audio_mfc" not in globals():
    load_audio_mfc = lambda: torch.randn(1, 80, 3000)

model = whisper.load_model("small", device="cpu") #tiny, base, small, medium, large, tiny.en, base.en, small.en, medium.en
model.requires_grad_(False)
model.eval()

tokenizer = whisper.decoding.get_tokenizer(
    model.is_multilingual,
    task="transcribe",
    language="en"
)

# x_mel shape: [batch, coeff=80, time=3000]
x_mel = load_audio_mfc()

# encode the audio
# x_audio shape: [batch, time=1500, feature=512]
start = time.time()
x_audio = model.encoder(x_mel)

# initialize using the start sequence
# x_tokens shape: [batch, seq<=448]
x_tokens = torch.tensor(
    [tokenizer.sot_sequence_including_notimestamps],
    dtype=torch.long,
)

max_tokens = 448
next_token = tokenizer.sot
while x_tokens.shape[1] <= max_tokens and next_token != tokenizer.eot:
    y_tokens = model.decoder(x_tokens, x_audio)

    next_token = y_tokens[0, -1].argmax()
    x_tokens = torch.concat(
        [x_tokens, next_token.reshape(1, 1)],
        axis=1,
    )

print("took", time.time() - start, "seconds")
print(tokenizer.decode(x_tokens[0]))

#: ONNX export
torch.onnx.export(
    model.encoder,
    (x_mel,),
    "encoder.onnx",
    input_names=["x"],
    output_names=["out"],
    dynamic_axes={
        "x": {0: "batch"},
        "out": {0: "batch"},
    },
)

torch.onnx.export(
    model.decoder,
    (x_tokens, x_audio),
    "decoder.onnx",
    input_names=["tokens", "audio"],
    output_names=["out"],
    dynamic_axes={
        "tokens": {0: "batch", 1: "seq"},
        "audio": {0: "batch"},
        "out": {0: "batch", 1: "seq"},
    },
)

#: Execute the ONNX model
import numpy as np
import onnxruntime

sess_encoder = onnxruntime.InferenceSession("encoder.onnx")
sess_decoder = onnxruntime.InferenceSession("decoder.onnx")

start = time.time()
out_encoder, = sess_encoder.run(["out"], {"x": x_mel.to('cpu').numpy()})

# initialize the tokens
tokens = list(tokenizer.sot_sequence_including_notimestamps)

next_token = tokenizer.sot
while x_tokens.shape[1] <= max_tokens and next_token != tokenizer.eot:
    out_decoder, = sess_decoder.run(
        ["out"],
        {
            "tokens": np.asarray([tokens], dtype="int64"),
            "audio": out_encoder,
        },
    )
    next_token = out_decoder[0, -1].argmax()
    tokens.append(next_token)

print("took", time.time() - start, "seconds")
print(tokenizer.decode(x_tokens[0]))

#: PyTorch with kv-caching
start = time.time()
whisper.decode(
    model,
    x_mel,
    options=whisper.DecodingOptions(
        fp16=False,
        without_timestamps=True,
        suppress_blank=False,
        suppress_tokens=[],
        language='english'
    ),
)
print(time.time() - start)

took 13.68833041191101 seconds
<|startoftranscript|><|en|><|transcribe|><|notimestamps|> [<|nospeech|><|endoftext|>


/usr/local/lib/python3.10/dist-packages/whisper/model.py:166: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert x.shape[1:] == self.positional_embedding.shape, "incorrect audio shape"


took 9.703990459442139 seconds
<|startoftranscript|><|en|><|transcribe|><|notimestamps|> [<|nospeech|><|endoftext|>
7.957705974578857
